### Configurações Pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}

25/03/13 20:48:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 20:48:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/13 20:48:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Configurações Pandas

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

### Lendo dados enriquecidos

In [3]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/banco_original_enriched')
           )

### Visit Occurrence

In [6]:
df_visit_occurrence = (df_input
                         .select('id_cidacs_mae_sinasc', 'dtnasc_sinasc', 'mesprenat_sinasc', 'person_id', 'codmunres_sinasc', 'visit_occurrence_id')
                        .withColumn('visit_occurrence_id', F.col('visit_occurrence_id'))
                        .withColumn('visit_concept_id', F.lit(9202))
                        .withColumn('visit_start_date', F.when(F.col('mesprenat_sinasc')<=9, F.add_months(F.col('dtnasc_sinasc'), -(9 - F.col('mesprenat_sinasc').cast('int'))))
                                 .when((F.col('mesprenat_sinasc')==99)|(F.col('mesprenat_sinasc').isNull()), F.lit('2099-12-31'))
                                 .when(F.col('mesprenat_sinasc')>9, F.trunc(F.col('dtnasc_sinasc'), 'month')))
                        .withColumn('visit_end_date', F.when(F.col('visit_start_date')=='2099-12-31', F.col('visit_start_date'))
                                                    .otherwise(F.date_add(F.col('visit_start_date'), 2)))
                        .withColumn('visit_type_concept_id', F.lit(32879))
                        .withColumn('care_site_id',  F.col('codmunres_sinasc'))
                        .withColumn('visit_start_datetime', F.lit(None))
                        .withColumn('visit_end_datetime', F.lit(None))
                        .withColumn('provider_id', F.lit(None))
                        .withColumn('visit_source_value', F.lit(None))
                        .withColumn('visit_source_concept_id', F.lit(None))
                        .withColumn('admitted_from_concept_id', F.lit(None))
                        .withColumn('admitted_from_source_value', F.lit(None))
                        .withColumn('discharged_to_concept_id', F.lit(None))
                        .withColumn('discharged_to_source_value', F.lit(None))
                        .withColumn('preceding_visit_occurrence_id', F.lit(None))
                    ).select(
                        F.col('visit_occurrence_id').cast('integer'),
                        F.col('person_id').cast('integer'),
                        F.col('visit_concept_id').cast('integer'),
                        F.col('visit_start_date').cast('date'),
                        F.col('visit_start_datetime').cast('timestamp'),
                        F.col('visit_end_date').cast('date'),
                        F.col('visit_end_datetime' ).cast('timestamp'),
                        F.col('visit_type_concept_id').cast('integer'),
                        F.col('provider_id').cast('integer'),
                        F.col('care_site_id').cast('integer'),
                        F.col('visit_source_value').cast('string'),
                        F.col('visit_source_concept_id').cast('integer'),
                        F.col('admitted_from_concept_id').cast('integer'),
                        F.col('admitted_from_source_value').cast('string'),
                        F.col('discharged_to_concept_id').cast('integer'),
                        F.col('discharged_to_source_value').cast('string'),
                        F.col('preceding_visit_occurrence_id').cast('integer')
                            )


df_visit_occurrence.count()

24696018

## Salvando no Postgresql

In [7]:
write_to_postgresl(df_visit_occurrence, tb_name='visit_occurrence', write_mode='append')

'Total time: 0:06:29.007400 and - Total rows: 24696018 - Total columns: 17'

### Deprecated

### Salvando CSV 

In [49]:
df_visit_occurrence.repartition(1).write.option("quoteAll",True).csv('/data/IDAF/SHARED/omop_tables_revisadas/visit_occurrence', header=True, mode='overwrite')

24/12/16 16:21:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/16 16:21:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/16 16:21:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/16 16:21:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


## SQL de insertion

In [ ]:
CREATE TABLE public.visit_occurrence (
			visit_occurrence_id integer NOT NULL,
			person_id integer NOT NULL,
			visit_concept_id integer NOT NULL,
			visit_start_date date NOT NULL,
			visit_start_datetime TIMESTAMP NULL,
			visit_end_date date NOT NULL,
			visit_end_datetime TIMESTAMP NULL,
			visit_type_concept_id Integer NOT NULL,
			provider_id integer NULL,
			care_site_id integer NULL,
			visit_source_value varchar(50) NULL,
			visit_source_concept_id integer NULL,
			admitted_from_concept_id integer NULL,
			admitted_from_source_value varchar(50) NULL,
			discharged_to_concept_id integer NULL,
			discharged_to_source_value varchar(50) NULL,
			preceding_visit_occurrence_id integer NULL );


CREATE TABLE public.visit_occurrence_pyspark (
			visit_occurrence_id varchar,
			person_id varchar,
			visit_concept_id varchar,
			visit_start_date varchar,
			visit_start_datetime varchar,
			visit_end_date varchar,
			visit_end_datetime varchar,
			visit_type_concept_id varchar,
			provider_id varchar,
			care_site_id varchar,
			visit_source_value varchar,
			visit_source_concept_id varchar,
			admitted_from_concept_id varchar,
			admitted_from_source_value varchar,
			discharged_to_concept_id varchar,
			discharged_to_source_value varchar,
			preceding_visit_occurrence_id varchar);


insert into public.visit_occurrence (
visit_occurrence_id ,
person_id ,
visit_concept_id ,
visit_start_date ,
visit_start_datetime ,
visit_end_date ,
visit_end_datetime ,
visit_type_concept_id ,
provider_id ,
care_site_id ,
visit_source_value ,
visit_source_concept_id ,
admitted_from_concept_id ,
admitted_from_source_value,
discharged_to_concept_id ,
discharged_to_source_value ,
preceding_visit_occurrence_id  
)
SELECT 
cast(visit_occurrence_id as integer) ,
cast(person_id as integer) ,
cast(visit_concept_id as integer) ,
cast(visit_start_date as date) ,
cast(case when visit_start_datetime='' then null else visit_start_datetime end as timestamp),
cast(visit_end_date as date) ,
cast(case when visit_end_datetime='' then null else visit_end_datetime end as timestamp),
cast(visit_type_concept_id as integer) ,
cast(case when provider_id='' then null else provider_id end as integer) ,
cast(case when care_site_id='' then null else care_site_id end as integer) ,
cast(case when visit_source_value='' then null else visit_source_value end as integer) ,
cast(case when visit_source_concept_id='' then null else visit_source_concept_id end as integer) ,
cast(case when admitted_from_concept_id='' then null else admitted_from_concept_id end as integer) ,
cast(admitted_from_source_value as varchar),
cast(case when discharged_to_concept_id='' then null else discharged_to_concept_id end as integer) ,
cast(discharged_to_source_value as varchar) ,
cast(case when preceding_visit_occurrence_id='' then null else preceding_visit_occurrence_id end as integer)
	FROM public.visit_occurrence_pyspark;
